# 🧠 AyseZeynepAhmet Depth Estimation - Colab Demo
This Colab notebook demonstrates monocular depth estimation using the **Probabilistic DepthNet** model with SAM-based masking.

You can run inference and visualize depth maps — **no installation needed!**

In [ ]:
# 🛠️ Install dependencies (PyTorch, OpenCV, etc.)
!pip install torch torchvision opencv-python numpy
!pip install git+https://github.com/facebookresearch/segment-anything.git


In [ ]:
# 📤 Upload a model checkpoint and a sample RGB image
from google.colab import files
uploaded = files.upload()  # Upload your .pth and image file


In [ ]:
# 📦 Import core libraries
import torch
import numpy as np
import cv2
from PIL import Image
from torchvision import transforms
from segment_anything import SamPredictor, sam_model_registry

# Visualization function
def colorize_depth(depth_tensor, min_depth=0.1, max_depth=100.0):
    if isinstance(depth_tensor, torch.Tensor):
        depth_tensor = depth_tensor.squeeze().cpu().numpy()
    depth_clipped = np.clip(depth_tensor, min_depth, max_depth)
    depth_norm = (depth_clipped - min_depth) / (max_depth - min_depth)
    depth_img = (depth_norm * 255).astype(np.uint8)
    return cv2.applyColorMap(depth_img, cv2.COLORMAP_INFERNO)


In [ ]:
# 🧠 Load model class
import torch.nn as nn
from torchvision.models import resnet18

class ProbabilisticDepthNet(nn.Module):
    def __init__(self, backbone='resnet18', min_depth=0.1, max_depth=100.0):
        super().__init__()
        self.min_depth = min_depth
        self.max_depth = max_depth
        encoder = resnet18(pretrained=True)
        self.encoder = nn.Sequential(*list(encoder.children())[:-2])
        self.depth_head = nn.Conv2d(512, 2, kernel_size=1)

    def forward(self, x):
        features = self.encoder(x)
        depth_params = self.depth_head(features)
        mean_raw, log_var = depth_params[:, 0:1, :, :], depth_params[:, 1:2, :, :]
        mean = torch.sigmoid(mean_raw) * (self.max_depth - self.min_depth) + self.min_depth
        var = torch.exp(log_var)
        mean = torch.nn.functional.interpolate(mean, size=x.shape[2:], mode='bilinear', align_corners=False)
        var = torch.nn.functional.interpolate(var, size=x.shape[2:], mode='bilinear', align_corners=False)
        return mean, var


In [ ]:
# 🔍 Run inference on uploaded image
# Replace below with actual uploaded filenames if needed
checkpoint_path = [f for f in uploaded if f.endswith('.pth')][0]
image_path = [f for f in uploaded if f.lower().endswith(('.jpg', '.png'))][0]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = ProbabilisticDepthNet().to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()

# Load image
img = Image.open(image_path).convert('RGB')
transform = transforms.Compose([
    transforms.Resize((192, 640)),
    transforms.ToTensor()
])
image_tensor = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    depth_pred, _ = model(image_tensor)

depth_color = colorize_depth(depth_pred)
cv2.imwrite("depth_colored.png", depth_color)
print("✅ Depth prediction complete.")


In [ ]:
# 📸 Show result
from IPython.display import Image as IPImage, display
display(IPImage("depth_colored.png"))
